In [1]:
import transformers
import re
from azure.communication.email import EmailClient


model_name = 'Intel/neural-chat-7b-v3-3'
model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

def generate_response(system_input, user_input):

    # Format the input using the provided template
    prompt = f"### System:\n{system_input}\n### User:\n{user_input}\n### Assistant:\n"

    # Tokenize and encode the prompt
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False)

    # Generate a response
    outputs = model.generate(inputs, max_length=10000, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the assistant's response
    return response.split("### Assistant:\n")[-1]



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/sdp/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
template = '''***official("Johnny showed unprofessional behavior at the beginning of the call by using inappropriate language and taking a long time, later he gave a contact information.")***'''

system_input = f'''You are the customer service supervisor. You should look at call transcripts and then see if there is any issue. if there is a issue in product you should say "{template}". if there is a issue with the customer service official you should say "***official("the problem description")***" . like what you say using the before rules must be enclosed with three asterisks before and after. the problem description between the parantheses be elaborate in explaining the problem and why it happened '''
user_input = '''[{'timestamp': (0.0, 3.08), 'text': ' Ellen speaking, how can I help you?'}, {'timestamp': (3.08, 5.36), 'text': ' Hello, this is Ryan Bartos.'}, {'timestamp': (5.36, 7.96), 'text': ' May I speak with Natalie Jones, please?'}, {'timestamp': (7.96, 11.2), 'text': " Can't you wait for a minute you dumb duck, I'll put you through."}, {'timestamp': (11.2, 15.08), 'text': " Mr. Bartos, I'm sorry, Natalie's in a meeting at the moment."}, {'timestamp': (15.08, 16.92), 'text': ' Would you like to leave a message?'}, {'timestamp': (16.92, 20.44), 'text': ' Yes, could you ask her to call me back as soon as possible?'}, {'timestamp': (20.44, 22.08), 'text': " It's pretty urgent."}, {'timestamp': (22.08, 24.52), 'text': ' Of course, does she have your number?'}, {'timestamp': (24.52, 28.18), 'text': ' She has my office number, but let me also give you my cell.'}, {'timestamp': (28.18, 33.46), 'text': " It's 472-555-8901."}, {'timestamp': (33.46, 35.34), 'text': ' Let me read that back to you.'}, {'timestamp': (35.34, 39.62), 'text': ' 472-555-8901.'}, {'timestamp': (39.62, 41.9), 'text': " That's right."}, {'timestamp': (41.9, 44.9), 'text': ' And could you spell your last name for me?'}, {'timestamp': (44.9, 46.5), 'text': ' B as in Boston.'}, {'timestamp': (46.5, 49.0), 'text': ' ARD as in Dog.'}, {'timestamp': (49.0, 51.5), 'text': ' O.S as in September.'}, {'timestamp': (51.5, 54.5), 'text': " Okay Mr. Bardo's, I'll give her the message."}, {'timestamp': (54.5, 56.5), 'text': ' Thanks a lot. Bye!'}]'''
# response = generate_response(system_input, user_input)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 ***official("Ryan Bartos showed unprofessional behavior at the beginning of the call by using inappropriate language. He later requested to speak with Natalie Jones, who was in a meeting. Ryan provided his office number and cell phone number for her to contact him. The agent took down the contact information and agreed to pass the message on to Natalie.")***


In [3]:
def prompt_remover(string):
    pattern = r'\*{3}(.*?)\*{3}'  
    return re.findall(pattern, string)

In [4]:
function = prompt_remover(response)

In [5]:
def extract_content(string):
    pattern = r'(\w+)\(\"(.*?)\"\)'
    match = re.match(pattern, string)
    if match:
        return match.groups()
    else:
        return None

In [6]:
ans = extract_content(function[0])

In [7]:
ans

('official',
 'Ryan Bartos showed unprofessional behavior at the beginning of the call by using inappropriate language. He later requested to speak with Natalie Jones, who was in a meeting. Ryan provided his office number and cell phone number for her to contact him. The agent took down the contact information and agreed to pass the message on to Natalie.')

In [13]:

def sendmail(recipent,subject,content):
    try:
        connection_string = "endpoint=https://intel-ai-hackathon.unitedstates.communication.azure.com/;accesskey=Jtk02urvGsZYOLJjT5xqjGYI9TMb0qLRwzFWvEHnzUMT9iiRhQ5lJ48HKoKEXTPAVLT1cWDU1vmQ/ymRQBnA7A=="
        client = EmailClient.from_connection_string(connection_string)

        message = {
            "senderAddress": "DoNotReply@07ddf0c0-e4bf-4093-8d55-396d8c1cbd30.azurecomm.net",
            "recipients":  {
                "to": [{"address": recipent }],
            },
            "content": {
                "subject": subject,
                "plainText": content,
            }
        }

        poller = client.begin_send(message)
        result = poller.result()

    except Exception as ex:
        print(ex)


In [16]:
def assess(user_input):
    template = '''***official("Johnny showed unprofessional behavior at the beginning of the call by using inappropriate language and taking a long time, later he gave a contact information.")***'''

    system_input = f'''You are the customer service supervisor. You should look at call transcripts and then see if there is any issue. if there is a issue in product you should say "{template}". if there is a issue with the customer service official you should say "***official("the problem description")***" . like what you say using the before rules must be enclosed with three asterisks before and after. the problem description between the parantheses be elaborate in explaining the problem and why it happened '''
    response = generate_response(system_input, user_input)
    queries = prompt_remover(response)
    for i in queries:
        task, content = extract_content(i)
        if (task=='official'):
            sendmail('nabothdemetrius@gmail.com', 'Issue regarding customer service', content)
            
            

In [17]:
assess(user_input)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [12]:
from azure.communication.email import EmailClient
